In [7]:
from pydantic.dataclasses import dataclass

@dataclass(frozen=True)
class A:
    id: int
    name: str
    
A(1, "A").__dict__



{'id': 1, 'name': 'A'}

In [60]:
from typing import TypeAlias, TypeVar, Generic, Callable, Any
from pydantic.dataclasses import dataclass

"""
This is based off of: https://github.com/rustedpy/result
"""

TOK = TypeVar("TOK")
TOKU = TypeVar("TOKU")
TERR = TypeVar("TERR")

@dataclass(frozen=True)
class Ok(Generic[TOK]):
    __match_args__ = ("_value",)
    _value: TOK
    def __init__(self, value: TOK): self._value = value
    def is_ok(self) -> bool: return True
    def is_err(self) -> bool: return False
    def unwrap(self) -> TOK: return self._value
    def unwrap_or(self, default: TOK) -> TOK: return self._value
    def map(self, f: Callable[[TOK], TOKU]) -> "Ok[TOKU]":
        return Ok(f(self._value))

@dataclass(frozen=True)
class Err(Generic[TERR]):
    __match_args__ = ("_err",)
    _err: TERR
    def __init__(self, err: TERR): self._err = err
    def is_ok(self) -> bool: return False
    def is_err(self) -> bool: return True
    def unwrap(self): raise Exception(f"Called unwrap on Err: {self._err}")
    def unwrap_or(self, default): return default
    def map(self, f: Callable[[Any], Any]) -> "Err[TERR]": return self

Result: TypeAlias = Ok[TOK] | Err[TERR]


from uuid import uuid4
def check_number_random() -> Result[int, str]:
    num = uuid4().int
    if  num % 2 == 0:
        return Ok(num)
    else:
        return Err("Not an even number")

# match check_number_random():
#     case Ok(value):
#         print(f"Even number: {value}")
#     case Err(err):
#         print(f"Error: {err}")
#     case _:
#         print("Unknown error")
res = check_number_random()
res.unwrap_or(0)

68626936063440590617763084572371833382

Even number: 303645041522558254169092523968029246142
